## 1. Rationale

* due to the sparseness, the ps is binned to 20 (can change) 
* then chisq test wheter the ratio of openning is dependent on bins 

##  2. load data

In [1]:
require(data.table)
dat.pro_ps<- fread("../dat/1901/alpha_beta.promoter.long_matrix_w_transcripts_ps.txt")

Loading required package: data.table


In [2]:
head(dat.pro_ps)
dim(dat.pro_ps)

gene_tr.idx,cluster,cell,ps
SAMD11_7,alpha_1,Islet1-fresh_AGACACCTAGGCAGAAGTAAGGAGCAGGA,5.643735
SAMD11_7,alpha_1,Islet1-fresh_AGACACCTATGCGCAGCGTCTAATGGTTG,1.056578
SAMD11_7,alpha_2,Islet1-fresh_AGACACCTCGAGGCTGAAGGCTATGGTTG,6.254863
SAMD11_7,alpha_2,Islet1-fresh_AGACACCTCGTACTAGCTAAGCCTGTACT,6.508574
SAMD11_7,alpha_1,Islet1-fresh_AGACACCTGGACTCCTTCGACTAGGGTTG,2.120670
SAMD11_7,alpha_2,Islet1-fresh_AGACACCTGGAGCTACAAGGAGTAAGGCG,5.564368


[1] 19412387        4

### 2.1 bin data 

In [4]:
require(tidyverse)
ps.range <- list()
ps.range$alpha <- range((dat.pro_ps %>%filter(cluster %in% c("alpha_1","alpha_2")))$ps)
ps.range$beta <- range((dat.pro_ps %>%filter(cluster %in% c("beta_1","beta_2")))$ps)
ps.range

$alpha
[1] 0.000000 6.746176

$beta
[1]  0.0000 20.2079

In [5]:
N.bin<- 101
ps.bks <- list()
ps.bks$alpha <- seq(ps.range$alpha[1],ps.range$alpha[2],length.out = N.bin)
ps.bks$beta <- seq(ps.range$beta[1],ps.range$beta[2],length.out = N.bin)

dat.pro_ps <- rbind(
    dat.pro_ps%>% 
    filter(cluster %in% c("alpha_1","alpha_2"))%>%
    mutate(ps_bin =cut(ps,breaks = ps.bks$alpha,include.lowest = T)),
    dat.pro_ps%>% 
    filter(cluster %in% c("beta_1","beta_2"))%>%
    mutate(ps_bin =cut(ps,breaks = ps.bks$beta,include.lowest = T))
)%>%separate(cluster,into=c("celltype","subtype"))

ps.bin.dat.sum<- dat.pro_ps%>%separate(cluster,into = c("celltype","subtype"))%>%
select(-one_of("gene_tr.idx","ps"))%>%distinct()

setDT(ps.bin.dat.sum)
ps.bin.dat.sum<-ps.bin.dat.sum[, .(.N), by = .(ps_bin,celltype)]
cat(paste(N.bin,"# bins:\n"))
cat("quantiles of alpha cell number per bin:\n")
t(quantile((ps.bin.dat.sum%>%filter(celltype=="alpha"))$N))
cat("quantiles of beta cell number per bin:\n")
t(quantile((ps.bin.dat.sum%>%filter(celltype=="beta"))$N))

101 # bins:
quantiles of alpha cell number per bin:


0%,25%,50%,75%,100%
1,21,41,71.25,257


quantiles of beta cell number per bin:


0%,25%,50%,75%,100%
2,18.75,33.5,77,655


In [6]:
ps.bin.dat.sum%>%group_by(celltype)%>%summarise(tot=sum(N))

celltype,tot
alpha,5594
beta,7170


In [13]:
head(ps.bin.dat.sum)
head(dat.pro_ps)
dim(dat.pro_ps)


ps_bin,celltype,N
"(5.6,5.67]",alpha,38
"(1.01,1.08]",alpha,151
"(6.21,6.27]",alpha,107
"(6.48,6.54]",alpha,19
"(2.09,2.16]",alpha,160
"(5.53,5.6]",alpha,28


gene_tr.idx,celltype,subtype,cell,ps,ps_bin
SAMD11_7,alpha,1,Islet1-fresh_AGACACCTAGGCAGAAGTAAGGAGCAGGA,5.643735,"(5.6,5.67]"
SAMD11_7,alpha,1,Islet1-fresh_AGACACCTATGCGCAGCGTCTAATGGTTG,1.056578,"(1.01,1.08]"
SAMD11_7,alpha,2,Islet1-fresh_AGACACCTCGAGGCTGAAGGCTATGGTTG,6.254863,"(6.21,6.27]"
SAMD11_7,alpha,2,Islet1-fresh_AGACACCTCGTACTAGCTAAGCCTGTACT,6.508574,"(6.48,6.54]"
SAMD11_7,alpha,1,Islet1-fresh_AGACACCTGGACTCCTTCGACTAGGGTTG,2.120670,"(2.09,2.16]"
SAMD11_7,alpha,2,Islet1-fresh_AGACACCTGGAGCTACAAGGAGTAAGGCG,5.564368,"(5.53,5.6]"


[1] 19412387        6

In [48]:
fwrite(dat.pro_ps,file = "../dat/1901/dat.pro_ps.csv")

## 3. run chisq test on dependence on bin

### 3.1 filter max fraction < 10%

In [16]:
ps.bin.dat.sum.2<-ps.bin.dat.sum%>%
    as.data.frame()%>%
    mutate(ps_bin =as.character(ps_bin))%>%
    column_to_rownames("ps_bin")

run_chisq_test<- function(g="SAMD11_7"){
    test.dat <- dat.sub%>%
    mutate(ps_bin=droplevels(ps_bin))%>%
    filter(gene_tr.idx=="SAMD11_7")%>%
    select(one_of("celltype","ps_bin"))

    test.dat.tab <- as.matrix(t(table(test.dat$ps_bin)))
    test.dat.tab<-rbind(test.dat.tab,ps.bin.dat.sum.2[colnames(test.dat.tab),'N']-test.dat.tab[1,])
    rownames(test.dat.tab)<- c("open","close")
    chisq <- chisq.test(test.dat.tab,simulate.p.value=T)
    chisq$p.value
}

In [64]:
require(parallel)
system.time(tmp <- lapply(c("alpha","beta"),function(ct){
    dat.sub <- dat.pro_ps%>%filter(celltype ==ct)
    all_tr <- unique(dat.sub$gene_tr.idx)
    mclapply(all_tr[1:10],function(g){
        run_chisq_test(g)
    },mc.cores = 10)
}))

   user  system elapsed 
  7.614   3.379   3.711 

In [65]:
tmp

[[1]]
[[1]][[1]]
[1] 0.2653673

[[1]][[2]]
[1] 0.2768616

[[1]][[3]]
[1] 0.2593703

[[1]][[4]]
[1] 0.2728636

[[1]][[5]]
[1] 0.2748626

[[1]][[6]]
[1] 0.2563718

[[1]][[7]]
[1] 0.2673663

[[1]][[8]]
[1] 0.2613693

[[1]][[9]]
[1] 0.2663668

[[1]][[10]]
[1] 0.2663668


[[2]]
[[2]][[1]]
[1] 0.2628686

[[2]][[2]]
[1] 0.2763618

[[2]][[3]]
[1] 0.2673663

[[2]][[4]]
[1] 0.2638681

[[2]][[5]]
[1] 0.2568716

[[2]][[6]]
[1] 0.2908546

[[2]][[7]]
[1] 0.2618691

[[2]][[8]]
[1] 0.2498751

[[2]][[9]]
[1] 0.2793603

[[2]][[10]]
[1] 0.2518741

In [51]:
names(res) <- c("alpha","beta")

In [57]:
tmp <- do.call(c,res$alpha)
names(tmp) <-  unique((dat.pro_ps%>%filter(celltype =="alpha"))$gene_tr.idx)
res$alpha <- tmp 

In [58]:
tmp <- do.call(c,res$beta)
names(tmp) <-  unique((dat.pro_ps%>%filter(celltype =="beta"))$gene_tr.idx)
res$beta <- tmp 

ERROR: Error in names(tmp) <- unique((dat.pro_ps %>% filter(celltype == "beta"))$gene_tr.idx): 'names' attribute [21825] must be the same length as the vector [21715]


In [59]:
head(tmp)

[1] 0.2583708 0.2568716 0.2738631 0.2663668 0.2513743 0.2678661

In [67]:
range(res$beta)
range(res$alpha)

[1] 0.2283858 0.3118441

[1] 0.2228886 0.3133433

In [63]:
head(unique((dat.pro_ps%>%filter(celltype =="alpha"))$gene_tr.idx))

[1] "SAMD11_7"   "SAMD11_13"  "SAMD11_14"  "NOC2L_19"   "KLHL17_23" 
[6] "PLEKHN1_27"